In [1]:
# mnist_cnn_deep
# MNIST and Convolutional Neural Network
# Conv,Maxpool : L1,L2,L3
# Fully Connected layer : L4,l5

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

In [2]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test,y_test) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [3]:
# one-hot 인코딩
nb_classes = 10     # 분류 class의 갯수(0 ~ 9)
Y_one_hot = tf.one_hot(y_train,nb_classes) # (60000, 10)
print(Y_one_hot.shape)                     # (60000, 10) , 2차원


# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
print(x_train.shape,x_train.dtype)  # 'float32'

x_test = tf.cast(x_test,dtype=tf.float32)
print(x_test.shape,x_test.dtype)  # 'float32


(60000, 10)
(60000, 28, 28) <dtype: 'float32'>
(10000, 28, 28) <dtype: 'float32'>


In [4]:
# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape)

(60000, 28, 28, 1)


In [5]:
# Layer 1  : conv2d - relu - max_pool
# (?,28,28,1) ---> (?,14,14,32)

# <1> conv2d
# L1 image shape : (?,28,28,1)
# filter : (3,3,1,32) , 필터 32개
# strides : (1,1,1,1), padding ='SAME'
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28,   --> (?,28,28,32)
W1 = tf.Variable(tf.random.normal([3,3,1,32]),name='weight1')  # filter
def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X))   # shape 변화가 없다

# <3> max_pool
# input image : (?,28,28,32)
# kernel size : (1,2,2,1), strides : (1,2,2,1), padding ='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14  ---> (?,14,14,32)
def L1_MaxPool(X):
    return tf.nn.max_pool(L1_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')    

In [6]:
# Layer2  : conv2d - relu - max_pool
# (?,14,14,32) --> (?,14,14,64) --> (?,7,7,64)

# <1> conv2d
# L2 image shape : (?,14,14,32)
# filter : (3,3,32,64) , 필터 64개
# strides : (1,1,1,1), padding ='SAME'
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14,   --> (?,14,14,64)
W2 = tf.Variable(tf.random.normal([3,3,32,64]),name='weight2')  # filter
def L2_conv2d(X):
    return tf.nn.conv2d(L1_MaxPool(X),W2,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X))   # shape 변화가 없다

# <3> max_pool
# input image : (?,14,14,64)
# kernel size : (1,2,2,1), strides : (1,2,2,1), padding ='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7  ---> (?,7,7,64)
def L2_MaxPool(X):
    return tf.nn.max_pool(L2_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

In [7]:
# Layer 3  : conv2d - relu - max_pool
# (?,7,7,64) --> (?,4*4*128)

# <1> conv2d
# L3 image shape : (?,7,7,64)
# filter : (3,3,64,128) , 필터 128개
# strides : (1,1,1,1), padding ='SAME'
# 출력 이미지 : (7+2 - 3)/1 + 1 = 7,   --> (?,7,7,128)
W3 = tf.Variable(tf.random.normal([3,3,64,128]),name='weight3')  # filter
def L3_conv2d(X):
    return tf.nn.conv2d(L2_MaxPool(X),W3,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L3_relu(X):
    return tf.nn.relu(L3_conv2d(X))   # shape 변화가 없다

# <3> max_pool
# input image : (?,7,7,128)
# kernel size : (1,2,2,1), strides : (1,2,2,1), padding ='SAME'
# 출력 이미지 : (7+1 - 2)/2 + 1 = 4  ---> (?,4,4,128)
def L3_MaxPool(X):
    return tf.nn.max_pool(L3_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') 

# <4> flatten layer  : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L3_flat(X):
    return tf.reshape(L3_MaxPool(X),[-1,4*4*128])

In [8]:
# Layer 4 : FC(Fully Connected Layer)
# (?,4*4*128) * (4*4*128,512 ) --> (?,512)

W4 = tf.Variable(tf.random.normal([4*4*128,512]), name ='weight4')
b4 = tf.Variable(tf.random.normal([512]), name = 'bias4')

def L4_relu(X):
    return  tf.nn.relu(tf.matmul(L3_flat(X),W4) + b4 ) 

In [9]:
# Layer 5 : FC(Fully Connected Layer)  
# L5 input shape : (?,512)
# (?,512) * (512,10) = (?,10)
nb_classes = 10
W5 = tf.Variable(tf.random.normal([512,nb_classes]), name ='weight5')
b5 = tf.Variable(tf.random.normal([nb_classes]), name = 'bias5')

In [10]:
# hypothesis 예측 함수 : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(L4_relu(X),W5) + b5

def hypothesis(X):
    return   tf.nn.softmax(logits(X))  

In [11]:
# 방법 2. batch 사이즈로 나누어 학습, 효율적 이며 학습 시간 단축
# 학습 시작

training_epochs = 100
batch_size = 600

# 경사 하강법
# learning_rate(학습율) 을 0.01로 설정하여 optimizer 객체를 생성
optimizer = tf.keras.optimizers.Adam(lr=0.01)

Y_one_hot = tf.one_hot(y_train,nb_classes)  # (60000,10)

print('***** Start Learning!!')
for epoch in range(training_epochs):  # 100
    
    avg_cost = 0
    
    # 100 = 60000/600
    total_batch = int(x_train.shape[0]/batch_size)
    for k in range(total_batch):  # 100 
        batch_xs = x_train[0+k*batch_size:batch_size + k*batch_size]
        batch_ys = Y_one_hot[0+k*batch_size:batch_size + k*batch_size]
        
        # X값의 shape을 4차원으로 변환 코드 추가
        X_img = tf.reshape(batch_xs,[-1,28,28,1])
        
        # 비용함수
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(X_img),
                                                     labels=batch_ys) 
            cost = tf.reduce_mean(cost_i)
            return cost    
        
        # cost를 minimize한다
        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,W4,b4,W5,b5])
        avg_cost += cost_func_batch().numpy()/total_batch
    
    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
          
print('***** Learning Finished')

***** Start Learning!!
Epoch: 0001 cost: 8717259.210312504
Epoch: 0002 cost: 229342.782148438
Epoch: 0003 cost: 124947.293085937
Epoch: 0004 cost: 76540.512119141
Epoch: 0005 cost: 52328.077875977
Epoch: 0006 cost: 35919.844736328
Epoch: 0007 cost: 25831.185454102
Epoch: 0008 cost: 20733.304230957
Epoch: 0009 cost: 16005.655946045
Epoch: 0010 cost: 13545.319007874
Epoch: 0011 cost: 11384.554515381
Epoch: 0012 cost: 8411.253893738
Epoch: 0013 cost: 6418.011741486
Epoch: 0014 cost: 4675.876184311
Epoch: 0015 cost: 3222.776914749
Epoch: 0016 cost: 2383.005056820
Epoch: 0017 cost: 2279.271544552
Epoch: 0018 cost: 1741.251465359
Epoch: 0019 cost: 982.688126769
Epoch: 0020 cost: 824.609064655
Epoch: 0021 cost: 1004.096433010
Epoch: 0022 cost: 531.598345668
Epoch: 0023 cost: 566.485647054
Epoch: 0024 cost: 588.316315575
Epoch: 0025 cost: 305.248266883
Epoch: 0026 cost: 281.842600479
Epoch: 0027 cost: 753.519858210
Epoch: 0028 cost: 336.408789978
Epoch: 0029 cost: 350.148099365
Epoch: 0030 cos

In [12]:
# 예측  , accuracy computation

def predict(X):
    return tf.argmax(hypothesis(X),1) 

Y_one_hot = tf.one_hot(y_test,nb_classes)  # (10000, 10)
# print(Y_one_hot.shape)

# X값의 shape을 4차원으로 변환 코드 추가
X_img = tf.reshape(x_test,[-1,28,28,1])

correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))

print('***** Predict')

pred = predict(X_img).numpy()
print(pred,y_test)
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.981

***** Predict
[7 2 1 ... 4 5 6] [7 2 1 ... 4 5 6]
Accuracy: 0.9873


In [13]:
# 정확도 비교

# [1] softmax 사용
# 1 layers              -------> Accuracy  : 0.8871
# 4 layers  sigmoid     -------> Accuracy  : 0.9033
# 4 layers  relu        -------> Accuracy  : 0.9534  

# [2] CNN 사용
# 3 layers              -------> Accuracy  : 0.981
# 5 layers              -------> Accuracy  : 0.9824 (epoch=50)
# 5 layers              -------> Accuracy  : 0.9873 (epoch=100)

# [3] Keras에서 CNN 사용
#
# VGGNet